# **PICO EXTRACTION** *| BERT NER*
### Participants/Problem (P), Intervention (I), Comparison (C) and Outcome (O)

In [1]:
# Intstall required libraries
!pip install datasets
!pip install -U accelerate
!pip install -U transformers
!pip install seqeval
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 5.9 MB/s eta 0:00:0000:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=d98b92aa8d279fe4e15a7560514c47322b39165c4c8b7b168b63b15eeed6ea60
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
# Import necessary libraries
import re
import pandas as pd
import numpy as np
import itertools
from datasets import Dataset
from datasets import load_metric
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
# Create necessary functions
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}

    
def replace_entity(old_entity):
    if "POPULATION" in old_entity or "SAMPLE" in old_entity:
        entity = "I-POPULATION"
    elif "PARTICIPANT" in old_entity:
        entity = "I-PARTICIPANT"
    elif "INTERVENTION" in old_entity:
        entity = "I-INTERVENTION"
    elif "COMPAR" in old_entity:
        entity = "I-COMPARISON"
    elif "OUTCOME" in old_entity:
        entity = "I-OUTCOME"
    elif bool(re.search('[a-zA-Z]', old_entity)) == False:
        entity = ""
    else:
        entity = "O"
    return(entity)

In [4]:
# Read the text data
# with open("/kaggle/input/pubmeddata/pubmed_data.txt") as f:
#     content = f.readlines()
    
with open("/kaggle/input/pubmeddata/pubmed_custom_picobert.txt") as f:
    content_custom = f.readlines()

In [5]:
# print(len(content))
print(len(content_custom))

71


### **DATA PREPARATION**
Convert the text data to PICO BERT trainable data format

Convert the text file and seperate the following fields in seperate column in a dataframe

* PMID
* PICO_Entity
* AOMRC Entity
* Text

In [6]:
print(content_custom)

['###27932540:\n', 'OBJECTIVE|A|on a reduction in the incidence of childhood obesity .\n', 'PARTICIPANTS|P|In school-age children .\n', 'INTERVENTIONS|I|what is the effect of a school-based physical activity program .\n', 'INTERVENTION AND COMPARISON|I|compared with no intervention .\n', '\n', '###27932541:\n', 'OBJECTIVE|A|on a reduction in reported incidences of bullying .\n', 'PARTICIPANTS|P|In high school children .\n', 'INTERVENTIONS|I|what is the effect of a nurse-led presentation on bullying .\n', 'INTERVENTION AND COMPARISON|I|ccompared with no intervention .\n', '\n', '###27932542:\n', 'OBJECTIVE|A|at an increased risk of developing esophageal cancer .\n', 'PARTICIPANTS|P|compared with males age @ and older .\n', 'INTERVENTIONS|I|who have a history of @ year of smoking or less .\n', 'INTERVENTION AND COMPARISON|I|who have no smoking history .\n', '\n', '###27932543:\n', 'OBJECTIVE|A|at greater risk for developing blood clots .\n', 'PARTICIPANTS|P|Are women ages @-@ .\n', 'INTE

In [7]:
# Data preprocessing & manipulation
# content_df = pd.DataFrame()
content_df_custom = pd.DataFrame()

# for i in tqdm(range(len(content))):
#     if("###" in content[i]):
#         content_df.loc[i, "PMID"] = content[i].split(":")[0].replace("#", "")
#     elif(content[i] == "\n"):
#         pass
#     else:
#         content_df.loc[i, "PMID"] = content_df.loc[i-1, "PMID"]
#         content_df.loc[i, "PICO_Entity"] = content[i].split("|")[0]
#         content_df.loc[i, "AOMRC_Entity"] = content[i].split("|")[1]
#         content_df.loc[i, "Text"] = content[i].split("|")[2]
for i in tqdm(range(len(content_custom))):
    if("###" in content_custom[i]):
        content_df_custom.loc[i, "PMID"] = content_custom[i].split(":")[0].replace("#", "")
    elif(content_custom[i] == "\n"):
        pass
    else:
        content_df_custom.loc[i, "PMID"] = content_df_custom.loc[i-1, "PMID"]
        content_df_custom.loc[i, "PICO_Entity"] = content_custom[i].split("|")[0]
        content_df_custom.loc[i, "AOMRC_Entity"] = content_custom[i].split("|")[1]
        content_df_custom.loc[i, "Text"] = content_custom[i].split("|")[2]

100%|██████████| 71/71 [00:00<00:00, 1496.96it/s]


In [8]:
# content_df.to_csv('processed_data.csv', encoding='utf-8')
content_df_custom.to_csv('processed_data_custom.csv', encoding='utf-8')

In [9]:
# Display manipulated data | samples
content_df = pd.read_csv("/kaggle/input/pubmeddata/processed_data.csv")
content_df = content_df.drop('Unnamed: 0', axis=1)
content_df.head()
content_df_custom.head()

,PMID,PICO_Entity,AOMRC_Entity,Text
0,27932540,NaN,NaN,NaN
1,27932540,OBJECTIVE,A,on a reduction in the incidence of childhood o...
2,27932540,PARTICIPANTS,P,In school-age children .\n
3,27932540,INTERVENTIONS,I,what is the effect of a school-based physical ...
4,27932540,INTERVENTION AND COMPARISON,I,compared with no intervention .\n


### **DATA CLEANING**
You can there are NAN's occupied with the columns

In [10]:
# Remove NAN's
content_df = content_df[pd.notnull(content_df['Text'])]
content_df_custom = content_df_custom[pd.notnull(content_df_custom['Text'])]

In [11]:
# Display cleaned data | samples
content_df.head()
content_df_custom.head()

,PMID,PICO_Entity,AOMRC_Entity,Text
1,27932540,OBJECTIVE,A,on a reduction in the incidence of childhood o...
2,27932540,PARTICIPANTS,P,In school-age children .\n
3,27932540,INTERVENTIONS,I,what is the effect of a school-based physical ...
4,27932540,INTERVENTION AND COMPARISON,I,compared with no intervention .\n
7,27932541,OBJECTIVE,A,on a reduction in reported incidences of bully...


### **DATA CONVERSION**
Convert the data with the required format and rename

In [12]:
# Datatype formating and renaming
content_df["Text"] = content_df["Text"].astype(str)
content_df["Text"] = content_df["Text"].apply(lambda x:x.strip())
content_df = content_df[["PMID", "Text", "PICO_Entity"]]
content_df.rename(columns = {'Text':'tokens', 'PICO_Entity':'ner_tags'}, inplace = True)
content_df = content_df.reset_index(drop=True)

# Datatype formating and renaming
content_df_custom["Text"] = content_df_custom["Text"].astype(str)
content_df_custom["Text"] = content_df_custom["Text"].apply(lambda x:x.strip())
content_df_custom = content_df_custom[["PMID", "Text", "PICO_Entity"]]
content_df_custom.rename(columns = {'Text':'tokens', 'PICO_Entity':'ner_tags'}, inplace = True)
content_df_custom = content_df_custom.reset_index(drop=True)

In [13]:
# Display formatted samples
content_df.head()
content_df_custom.head()

,PMID,tokens,ner_tags
0,27932540,on a reduction in the incidence of childhood o...,OBJECTIVE
1,27932540,In school-age children .,PARTICIPANTS
2,27932540,what is the effect of a school-based physical ...,INTERVENTIONS
3,27932540,compared with no intervention .,INTERVENTION AND COMPARISON
4,27932541,on a reduction in reported incidences of bully...,OBJECTIVE


### **DATA PREPROCESSING**
Convert the data required parameters that needs to train the model

In [14]:
# Calculate token length
content_df["token_len"] = content_df["tokens"].apply(lambda x: len(x.split()))

content_df_custom["token_len"] = content_df_custom["tokens"].apply(lambda x: len(x.split()))

In [15]:
# Display samples
print(content_df.head())
print(content_df_custom.head())

       PMID                                             tokens  \
0  28628768  To test the efficacy of a pregnancy adapted ve...   
1  28628768                             Online and telephone .   
2  28628768  Self-referred pregnant women ( gestational wee...   
3  28628768  @ pregnant women ( gestational week @ ) with m...   
4  28628768  The primary outcome was depressive symptoms me...   

                ner_tags  token_len  
0              OBJECTIVE         29  
1                SETTING          4  
2   POPULATION OR SAMPLE         17  
3                METHODS         36  
4  MAIN OUTCOME MEASURES         18  
       PMID                                             tokens  \
0  27932540  on a reduction in the incidence of childhood o...   
1  27932540                           In school-age children .   
2  27932540  what is the effect of a school-based physical ...   
3  27932540                    compared with no intervention .   
4  27932541  on a reduction in reported incid

In [16]:
# Replace entities with required entity names
content_df["ner_tags"] = content_df["ner_tags"].apply(lambda x: replace_entity(x))

# Replace entities with required entity names
content_df_custom["ner_tags"] = content_df_custom["ner_tags"].apply(lambda x: replace_entity(x))

In [17]:
# Display samples
print(content_df.head())
print(content_df_custom.head())

       PMID                                             tokens      ner_tags  \
0  28628768  To test the efficacy of a pregnancy adapted ve...             O   
1  28628768                             Online and telephone .             O   
2  28628768  Self-referred pregnant women ( gestational wee...  I-POPULATION   
3  28628768  @ pregnant women ( gestational week @ ) with m...             O   
4  28628768  The primary outcome was depressive symptoms me...     I-OUTCOME   

   token_len  
0         29  
1          4  
2         17  
3         36  
4         18  
       PMID                                             tokens  \
0  27932540  on a reduction in the incidence of childhood o...   
1  27932540                           In school-age children .   
2  27932540  what is the effect of a school-based physical ...   
3  27932540                    compared with no intervention .   
4  27932541  on a reduction in reported incidences of bully...   

         ner_tags  token_len  
0

In [18]:
# Iterate entities to all words/token
for i in range(len(content_df)):
    content_df.loc[i, "ner_tags"] = str([content_df["ner_tags"][i]] * content_df["token_len"][i])
    content_df.loc[i, "tokens"] = str(content_df["tokens"][i].split())

# Iterate entities to all words/token
for i in range(len(content_df_custom)):
    content_df_custom.loc[i, "ner_tags"] = str([content_df_custom["ner_tags"][i]] * content_df_custom["token_len"][i])
    content_df_custom.loc[i, "tokens"] = str(content_df_custom["tokens"][i].split())

In [19]:
content_df.head()
content_df_custom.head()

,PMID,tokens,ner_tags,token_len
0,27932540,"['on', 'a', 'reduction', 'in', 'the', 'inciden...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",10
1,27932540,"['In', 'school-age', 'children', '.']","['I-PARTICIPANT', 'I-PARTICIPANT', 'I-PARTICIP...",4
2,27932540,"['what', 'is', 'the', 'effect', 'of', 'a', 'sc...","['I-INTERVENTION', 'I-INTERVENTION', 'I-INTERV...",11
3,27932540,"['compared', 'with', 'no', 'intervention', '.']","['I-INTERVENTION', 'I-INTERVENTION', 'I-INTERV...",5
4,27932541,"['on', 'a', 'reduction', 'in', 'reported', 'in...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']",9


In [20]:
# # Converts string to list of values
content_df["ner_tags"] = content_df["ner_tags"].apply(lambda x:eval(x))
content_df["tokens"] = content_df["tokens"].apply(lambda x:eval(x))
\
# Converts string to list of values
content_df_custom["ner_tags"] = content_df_custom["ner_tags"].apply(lambda x:eval(x))
content_df_custom["tokens"] = content_df_custom["tokens"].apply(lambda x:eval(x))

In [21]:
# # Group by PMID
content_df = content_df.groupby('PMID').agg(lambda x: x.to_list())

# Group by PMID
content_df_custom = content_df_custom.groupby('PMID').agg(lambda x: x.to_list())

In [22]:
# Combine list of list
content_df["tokens"] = content_df["tokens"].apply(lambda x:list(itertools.chain.from_iterable(x)))
content_df["ner_tags"] = content_df["ner_tags"].apply(lambda x:list(itertools.chain.from_iterable(x)))

# Combine list of list
content_df_custom["tokens"] = content_df_custom["tokens"].apply(lambda x:list(itertools.chain.from_iterable(x)))
content_df_custom["ner_tags"] = content_df_custom["ner_tags"].apply(lambda x:list(itertools.chain.from_iterable(x)))

In [23]:
condition = (content_df.index == 1281030) | (content_df.index == 1283117) | (content_df.index == 1283730) | (content_df.index == 1283984) | (content_df.index == 1286524)
content_df_custom_test = content_df.loc[condition]
content_df = content_df.loc[~condition]

In [24]:
# Reset dataframe
content_df = content_df.reset_index(drop=True)
# Reset dataframe
content_df_custom_test = content_df_custom_test.reset_index(drop=True)
# Reset dataframe
content_df_custom = content_df_custom.reset_index(drop=True)

In [25]:
# # Display preprocessed data | samples
content_df.head()

content_df_custom.head()

,tokens,ner_tags,token_len
0,"[on, a, reduction, in, the, incidence, of, chi...","[O, O, O, O, O, O, O, O, O, O, I-PARTICIPANT, ...","[10, 4, 11, 5]"
1,"[on, a, reduction, in, reported, incidences, o...","[O, O, O, O, O, O, O, O, O, I-PARTICIPANT, I-P...","[9, 5, 11, 5]"
2,"[at, an, increased, risk, of, developing, esop...","[O, O, O, O, O, O, O, O, O, I-PARTICIPANT, I-P...","[9, 8, 12, 6]"
3,"[at, greater, risk, for, developing, blood, cl...","[O, O, O, O, O, O, O, O, I-PARTICIPANT, I-PART...","[8, 5, 5, 6]"
4,"[more, effective, in, detecting, breast, cance...","[O, O, O, O, O, O, O, I-PARTICIPANT, I-PARTICI...","[7, 6, 5, 8]"


In [26]:
# Save preprocessed data
content_df.to_excel("preprocessed_df.xlsx")

content_df_custom.to_excel("preprocessed_df_custom.xlsx")

**TEST / TRAIN SPLIT**
---
Split the train data set into two sets by the ratio 8:2. and we can furthur evaluate the model performance

In [27]:
#content_df = pd.read_excel("/kaggle/input/pubmeddata/preprocessed_df_complete.xlsx")

#content_df_custom = pd.read_excel("/kaggle/working/preprocessed_df_custom.xlsx")

In [28]:
content_df.head()

,tokens,ner_tags,token_len
0,"[The, effect, of, selective, decontamination, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[28, 16, 42, 7, 23, 16, 11, 26, 13, 9, 25, 23]"
1,"[To, investigate, prospectively, the, effects,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[20, 16, 15, 10, 26, 37, 13, 5, 24, 19, 30, 12]"
2,"[To, evaluate, clinically, the, accuracy, of, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[19, 4, 12, 5, 16, 29, 23, 16, 37, 27]"
3,"[To, assess, the, accuracy, of, room-temperatu...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[16, 18, 21, 11, 13, 26, 14, 8, 18, 20, 27, 29]"
4,"[To, investigate, the, influence, of, ocean, s...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[34, 19, 14, 19, 31, 20, 18, 25, 36, 10, 24]"


In [29]:
#Test train split
train, test = train_test_split(content_df, test_size=0.2, shuffle=False, random_state=1)

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
content_df_custom = content_df_custom.reset_index(drop=True)
content_df_custom_test = content_df_custom_test.reset_index(drop=True)

train_df = train
test_df = test
test_df_custom = content_df_custom
test_df_custom_test = content_df_custom_test

In [30]:
# Print Shape
print(len(train_df))
print(len(test_df))

# Print Shape
print(len(test_df_custom))

19730
4933
7


In [31]:
# Label to numeric representation
label_list = ['O','I-POPULATION', 'I-PARTICIPANT','I-INTERVENTION', 'I-COMPARISON','I-OUTCOME']
label_encoding_dict = {'O': 0, 'I-POPULATION': 1, 'I-PARTICIPANT': 2, 'I-INTERVENTION': 3, 'I-COMPARISON': 4, 'I-OUTCOME': 5}

**MODEL INITIALIZATION**
---
Initialize the BERT model. Define the Task Name, Model, Tokenizer

In [51]:
# Initialize the BERT model
task = "ner" 
model_checkpoint = "distilbert-base-uncased"
batch_size = 32
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

# Initialize pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) 

data_collator = DataCollatorForTokenClassification(tokenizer) #Some random data enhancements

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream

**TOKENIZATION**
---
Tokenize and embed the dataset with pretrained BERT

In [52]:
train_df

,tokens,ner_tags,token_len
0,"[The, effect, of, selective, decontamination, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[28, 16, 42, 7, 23, 16, 11, 26, 13, 9, 25, 23]"
1,"[To, investigate, prospectively, the, effects,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[20, 16, 15, 10, 26, 37, 13, 5, 24, 19, 30, 12]"
2,"[To, evaluate, clinically, the, accuracy, of, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[19, 4, 12, 5, 16, 29, 23, 16, 37, 27]"
3,"[To, assess, the, accuracy, of, room-temperatu...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[16, 18, 21, 11, 13, 26, 14, 8, 18, 20, 27, 29]"
4,"[To, investigate, the, influence, of, ocean, s...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[34, 19, 14, 19, 31, 20, 18, 25, 36, 10, 24]"
...,...,...,...
19725,"[The, objective, of, this, study, was, to, det...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[38, 30, 31, 13, 17, 17, 21, 22, 21, 26, 14]"
19726,"[hospital, falls, place, a, substantial, burde...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[10, 18, 19, 21, 9, 16, 12, 8, 15, 18, 17, 30,..."
19727,"[Optimal, bowel, preparation, is, associated, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[24, 29, 9, 10, 36, 22, 21, 29, 39, 28, 19, 17..."
19728,"[Manual, in-line, stabilisation, is, usually, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[26, 18, 28, 6, 17, 14, 29, 6, 12, 30, 11, 18,..."


In [53]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
test_dataset_custom = Dataset.from_pandas(test_df_custom)
test_dataset_custom_test = Dataset.from_pandas(test_df_custom_test)

train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets_custom = test_dataset_custom.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets_custom_test = test_dataset_custom_test.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

**HYPER PARAMETER TURNING**
---
Hyperparameters - We can iterate and tune the model with these parameters for better results

In [54]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=6,
    weight_decay=1e-5,
)

metric = load_metric("seqeval")

**TRAINING MODULE**
---
Train the model with different parameters and finalize the optimal one

In [55]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.297610,0.286193,0.521364,0.369536,0.881512
2,0.292800,0.268182,0.326624,0.534493,0.405469,0.897232
3,0.292800,0.268491,0.369349,0.620062,0.462941,0.904385
4,0.166600,0.309057,0.370907,0.666590,0.476614,0.902739
5,0.094000,0.314355,0.404117,0.653346,0.499362,0.908570
6,0.094000,0.350243,0.393829,0.635034,0.486158,0.907937


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=1854, training_loss=0.1596019769563644, metrics={'train_runtime': 3764.696, 'train_samples_per_second': 31.445, 'train_steps_per_second': 0.492, 'total_flos': 1.5467144590296288e+16, 'train_loss': 0.1596019769563644, 'epoch': 6.0})

# **EVALUATION**

---

Evaluate the custom dataset

In [37]:
trainer.predict(test_dataset= test_tokenized_datasets_custom)

/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[[ 2.432476  , -0.9382089 , -0.25086045, -0.8915843 ,
         -3.3387039 ,  1.346985  ],
        [ 5.3726745 , -1.8365083 , -1.819191  , -2.0922387 ,
         -4.718863  ,  1.0336202 ],
        [ 5.3672266 , -1.793717  , -1.8081288 , -2.235689  ,
         -4.783045  ,  1.0629711 ],
        ...,
        [ 1.6977676 , -2.6276307 , -1.0824155 ,  1.5617069 ,
         -4.3974385 ,  1.6647342 ],
        [ 1.7162296 , -2.7068176 , -1.0737661 ,  1.6736349 ,
         -4.5496087 ,  1.7012951 ],
        [ 1.7911983 , -2.6876152 , -1.1868043 ,  1.4637051 ,
         -4.6379123 ,  1.9388874 ]],

       [[ 2.701241  , -0.5868128 , -0.54619575, -1.0346653 ,
         -3.1787145 ,  1.4109614 ],
        [ 4.431157  , -1.6349794 , -1.7150513 , -1.7081288 ,
         -4.7965574 ,  1.8495064 ],
        [ 4.511697  , -1.5988492 , -1.7269357 , -1.8684956 ,
         -4.886233  ,  1.767313  ],
        ...,
        [ 2.0427754 , -1.9158728 , -1.2725607 ,  0.7523157 ,
         

In [38]:
trainer.evaluate(test_tokenized_datasets_custom)

{'eval_loss': 1.804616928100586,
 'eval_precision': 0.17647058823529413,
 'eval_recall': 0.125,
 'eval_f1': 0.14634146341463414,
 'eval_accuracy': 0.44473684210526315,
 'eval_runtime': 0.087,
 'eval_samples_per_second': 80.434,
 'eval_steps_per_second': 11.491,
 'epoch': 3.0}

In [56]:
trainer.evaluate(test_tokenized_datasets_custom_test)

/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.3325570523738861,
 'eval_precision': 0.4,
 'eval_recall': 1.0,
 'eval_f1': 0.5714285714285715,
 'eval_accuracy': 0.9211892402076451,
 'eval_runtime': 0.19,
 'eval_samples_per_second': 26.322,
 'eval_steps_per_second': 5.264,
 'epoch': 6.0}

*Save the Model*
---
Save the model to disk

In [39]:
# Save the model to disk
trainer.save_model('un-ner.model')

**PREDICTION MODULE**
---
Predict the **Unseen Data**


In [40]:
# Load the tokenizer and trained model from disk
tokenizer = AutoTokenizer.from_pretrained('./un-ner.model/')
model = AutoModelForTokenClassification.from_pretrained('./un-ner.model/', num_labels=len(label_list))

In [41]:
# Input unseen data
paragraph = '''
OBJECTIVE|A|reduce the future risk of stroke .
PARTICIPANTS|P|For women under age @ .
INTERVENTIONS|I|does the daily use of @ mg low-dose Aspirin .
INTERVENTION AND COMPARISON|I|compared with no usage of low-dose Aspirin .
'''

In [42]:
# Tokenize the paragraph
tokens = tokenizer(paragraph)
torch.tensor(tokens['input_ids']).unsqueeze(0).size()

torch.Size([1, 59])

In [43]:
# Predictions
predictions = model.forward(input_ids=torch.tensor(tokens['input_ids']).unsqueeze(0), attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))
predictions = torch.argmax(predictions.logits.squeeze(), axis=1)
predictions = [label_list[i] for i in predictions]

In [44]:
print(predictions)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-INTERVENTION', 'O', 'I-INTERVENTION', 'O', 'I-INTERVENTION', 'I-INTERVENTION', 'I-INTERVENTION', 'I-INTERVENTION', 'O', 'I-INTERVENTION', 'I-INTERVENTION', 'I-INTERVENTION', 'I-INTERVENTION', 'O', 'I-OUTCOME', 'I-OUTCOME', 'I-OUTCOME', 'I-OUTCOME', 'I-OUTCOME', 'I-OUTCOME', 'I-OUTCOME', 'I-OUTCOME', 'I-OUTCOME', 'O', 'O', 'O', 'O', 'O', 'O', 'I-OUTCOME', 'I-OUTCOME', 'I-OUTCOME', 'I-OUTCOME']


In [45]:
words = tokenizer.batch_decode(tokens['input_ids'])

In [46]:
# Store the token and predicted entities in a dataframe
prediction_results = pd.DataFrame({'ner': predictions, 'words': words})

In [47]:
# Print predicted samples
print(prediction_results)

               ner          words
0                O          [CLS]
1                O      objective
2                O              |
3                O              a
4                O              |
5                O         reduce
6                O            the
7                O         future
8                O           risk
9                O             of
10               O         stroke
11               O              .
12               O   participants
13               O              |
14               O              p
15               O              |
16               O            for
17               O          women
18               O          under
19               O            age
20               O              @
21               O              .
22               O  interventions
23               O              |
24               O              i
25               O              |
26  I-INTERVENTION           does
27               O            the
28  I-INTERVEN

*Export the results*
---
Export test results to **csv**

Select the **Words, Labels** and **Confidence scores** to export csv file

In [48]:
label_list = ['O','I-POPULATION', 'I-PARTICIPANT','I-INTERVENTION', 'I-COMPARISON','I-OUTCOME']
O_List=[]
I_POPULATION_List=[]
I_PARTICIPANT_List=[]
I_INTERVENTION_List=[]
I_COMPARISON_List=[]
I_OUTCOME_List=[]
for ner,word in zip(predictions,words):
    if ner == 'O':
        O_List.append(word)
    elif ner == 'I-POPULATION':
        I_POPULATION_List.append(word)
    elif ner == 'I-PARTICIPANT':
        I_PARTICIPANT_List.append(word)
    elif ner == 'I-INTERVENTION':
        I_INTERVENTION_List.append(word)
    elif ner == 'I-COMPARISON':
        I_COMPARISON_List.append(word)
    elif ner == 'I-OUTCOME':
        I_OUTCOME_List.append(word)

In [49]:
print('O       '+' '.join(O_List))
print("------------------------------------------")
print('I-POPULATION        ' + ' '.join(I_POPULATION_List))
print("------------------------------------------")
print('I-PARTICIPANT       '+' '.join(I_PARTICIPANT_List))
print("------------------------------------------")
print('I-INTERVENTION       '+' '.join(I_INTERVENTION_List))
print("------------------------------------------")
print('I-COMPARISON        ' + ' '.join(I_COMPARISON_List))
print("------------------------------------------")
print('I-OUTCOME        ' + ' '.join(I_OUTCOME_List))

O       [CLS] objective | a | reduce the future risk of stroke . participants | p | for women under age @ . interventions | i | the use - . usage of low - dose as
------------------------------------------
I-POPULATION        
------------------------------------------
I-PARTICIPANT       
------------------------------------------
I-INTERVENTION       does daily of @ mg low dose as ##pi ##rin
------------------------------------------
I-COMPARISON        
------------------------------------------
I-OUTCOME        intervention and comparison | i | compared with no ##pi ##rin . [SEP]


In [50]:
# Export results to csv
prediction_results.to_csv("prediction_results.csv")